In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169#, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.utils import multi_gpu_model
import tensorflow as tf

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics

Using TensorFlow backend.


In [2]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [3]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [4]:
y = y.set_index("image_id")

In [5]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [6]:
size=(137,137)

def transformImg(img):
    img=255-img
    mu = cv2.moments(img, False)
    x, y= mu["m10"]/mu["m00"] , mu["m01"]/mu["m00"]
    M = np.float32([[1,0,(236/2)-x],[0,1,(137/2)-y]])
    img = cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    img = cv2.filter2D(img,-1,kernel)
    img = cv2.resize(img, (size[0],size[1]))
    #img = preprocess_input(img)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [7]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, i):
        _imgs=self.X[i * self.batch_size:(i + 1) * self.batch_size,:,:]
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img))
        
        imgs=np.asarray(imgs).astype(np.float32)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[i * self.batch_size:(i + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training and np.random.random()<0.8:
        #if self.training:
            r= np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            alpha=np.random.rand(imgs.shape[0])
            imgs=np.tile(alpha,(3,*size,1)).T*imgs+np.tile((1-alpha),(3,*size,1)).T*imgs2
            grapheme=np.tile(alpha,(168,1)).T*grapheme+np.tile((1-alpha),(168,1)).T*grapheme2
            vowel=np.tile(alpha,(11,1)).T*vowel+np.tile((1-alpha),(11,1)).T*vowel2
            consonant=np.tile(alpha,(7,1)).T*consonant+np.tile((1-alpha),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
            
        if self.training:
            imgs = [randomErase(img) for img in imgs]
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [8]:
def getResBased(cat=168):
    model =  ResNet152V2(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(cat, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getDenseBased(cat=168):
    model =  DenseNet121(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(cat, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getMultiResBased():
    model =  ResNet152V2(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

def getMultiDenseBased():
    model =  DenseNet169(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model


def getMultiXceptionBased():
    model =  Xception(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model


def getMultiMobileBased():
    model =  MobileNetV2(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)

In [10]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="weightsXceptiontwice-epoch{epoch:04}.h5")

In [11]:
train_gen = DataLoader(X_train, y_train, training=True, batch_size=64)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=64)
model = getMultiXceptionBased()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[2.,1.,1.])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=40, callbacks=[early_stopping, checkpoint], workers=multiprocessing.cpu_count(), use_multiprocessing=True)
model.save("multixception.h5")

compiled
Epoch 1/40
2825/2825 [==============================] - 1076s 381ms/step - loss: 7.1291 - dense_2_loss: 2.7293 - dense_3_loss: 1.0345 - dense_4_loss: 0.6361 - dense_2_acc: 0.5048 - dense_3_acc: 0.7675 - dense_4_acc: 0.8406 - val_loss: 1.6748 - val_dense_2_loss: 0.8316 - val_dense_3_loss: 0.3891 - val_dense_4_loss: 0.1720 - val_dense_2_acc: 0.7964 - val_dense_3_acc: 0.9025 - val_dense_4_acc: 0.9585
Epoch 2/40
2825/2825 [==============================] - 1045s 370ms/step - loss: 5.5620 - dense_2_loss: 2.0925 - dense_3_loss: 0.8487 - dense_4_loss: 0.5282 - dense_2_acc: 0.6898 - dense_3_acc: 0.8304 - dense_4_acc: 0.8831 - val_loss: 1.3055 - val_dense_2_loss: 0.4691 - val_dense_3_loss: 0.1784 - val_dense_4_loss: 0.1361 - val_dense_2_acc: 0.8718 - val_dense_3_acc: 0.9555 - val_dense_4_acc: 0.9626
Epoch 3/40
 182/2825 [>.............................] - ETA: 15:42 - loss: 6.1834 - dense_2_loss: 2.3111 - dense_3_loss: 0.9552 - dense_4_loss: 0.6060 - dense_2_acc: 0.6937 - dense_3_acc: 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1044s 369ms/step - loss: 5.1805 - dense_2_loss: 1.9454 - dense_3_loss: 0.7888 - dense_4_loss: 0.5009 - dense_2_acc: 0.7154 - dense_3_acc: 0.8397 - dense_4_acc: 0.8876 - val_loss: 0.5133 - val_dense_2_loss: 0.3807 - val_dense_3_loss: 0.1320 - val_dense_4_loss: 0.1082 - val_dense_2_acc: 0.8985 - val_dense_3_acc: 0.9677 - val_dense_4_acc: 0.9721
Epoch 4/40
2825/2825 [==============================] - 1046s 370ms/step - loss: 4.9271 - dense_2_loss: 1.8464 - dense_3_loss: 0.7514 - dense_4_loss: 0.4830 - dense_2_acc: 0.7264 - dense_3_acc: 0.8416 - dense_4_acc: 0.8913 - val_loss: 0.6963 - val_dense_2_loss: 0.3392 - val_dense_3_loss: 0.1303 - val_dense_4_loss: 0.1232 - val_dense_2_acc: 0.9088 - val_dense_3_acc: 0.9707 - val_dense_4_acc: 0.9715
Epoch 5/40
2825/2825 [==============================] - 1045s 370ms/step - loss: 4.7127 - dense_2_loss: 1.7607 - dense_3_loss: 0.7234 - dense_4_loss: 0.4679 - dense_2_acc: 0.7342 - dense_3_acc: 0.8468 - dense_

2825/2825 [==============================] - 1042s 369ms/step - loss: 3.2615 - dense_2_loss: 1.1678 - dense_3_loss: 0.5535 - dense_4_loss: 0.3724 - dense_2_acc: 0.7902 - dense_3_acc: 0.8955 - dense_4_acc: 0.9260 - val_loss: 0.2640 - val_dense_2_loss: 0.1593 - val_dense_3_loss: 0.0547 - val_dense_4_loss: 0.0482 - val_dense_2_acc: 0.9594 - val_dense_3_acc: 0.9874 - val_dense_4_acc: 0.9880
Epoch 24/40
2825/2825 [==============================] - 1045s 370ms/step - loss: 3.2286 - dense_2_loss: 1.1541 - dense_3_loss: 0.5500 - dense_4_loss: 0.3703 - dense_2_acc: 0.7903 - dense_3_acc: 0.8979 - dense_4_acc: 0.9263 - val_loss: 0.2999 - val_dense_2_loss: 0.1774 - val_dense_3_loss: 0.0573 - val_dense_4_loss: 0.0488 - val_dense_2_acc: 0.9536 - val_dense_3_acc: 0.9864 - val_dense_4_acc: 0.9874
Epoch 25/40
2825/2825 [==============================] - 1042s 369ms/step - loss: 3.1956 - dense_2_loss: 1.1382 - dense_3_loss: 0.5480 - dense_4_loss: 0.3711 - dense_2_acc: 0.7928 - dense_3_acc: 0.8981 - dens

In [ ]:
model.fit_generator(train_gen, validation_data=valid_gen, epochs=40, callbacks=[early_stopping, checkpoint], workers=multiprocessing.cpu_count(), use_multiprocessing=True)
model.save("multixception.h5")

Epoch 1/40
2825/2825 [==============================] - 1055s 374ms/step - loss: 2.8670 - dense_2_loss: 1.0038 - dense_3_loss: 0.5122 - dense_4_loss: 0.3472 - dense_2_acc: 0.8128 - dense_3_acc: 0.9104 - dense_4_acc: 0.9350 - val_loss: 0.3762 - val_dense_2_loss: 0.1607 - val_dense_3_loss: 0.0469 - val_dense_4_loss: 0.0443 - val_dense_2_acc: 0.9619 - val_dense_3_acc: 0.9891 - val_dense_4_acc: 0.9890
Epoch 1/40Epoch 2/40
2825/2825 [==============================] - 1051s 372ms/step - loss: 2.8408 - dense_2_loss: 0.9933 - dense_3_loss: 0.5095 - dense_4_loss: 0.3447 - dense_2_acc: 0.8146 - dense_3_acc: 0.9113 - dense_4_acc: 0.9366 - val_loss: 0.1502 - val_dense_2_loss: 0.1682 - val_dense_3_loss: 0.0472 - val_dense_4_loss: 0.0478 - val_dense_2_acc: 0.9583 - val_dense_3_acc: 0.9886 - val_dense_4_acc: 0.9880
Epoch 3/40
2825/2825 [==============================] - 1046s 370ms/step - loss: 2.8341 - dense_2_loss: 0.9913 - dense_3_loss: 0.5076 - dense_4_loss: 0.3439 - dense_2_acc: 0.8153 - dense_3

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 949s 336ms/step - loss: 2.7576 - dense_2_loss: 0.9589 - dense_3_loss: 0.5017 - dense_4_loss: 0.3382 - dense_2_acc: 0.8207 - dense_3_acc: 0.9142 - dense_4_acc: 0.9378 - val_loss: 0.1551 - val_dense_2_loss: 0.1511 - val_dense_3_loss: 0.0467 - val_dense_4_loss: 0.0459 - val_dense_2_acc: 0.9621 - val_dense_3_acc: 0.9897 - val_dense_4_acc: 0.9886
Epoch 10/40
1871/2825 [==================>...........] - ETA: 5:10 - loss: 2.7826 - dense_2_loss: 0.9643 - dense_3_loss: 0.5106 - dense_4_loss: 0.3433 - dense_2_acc: 0.8231 - dense_3_acc: 0.9169 - dense_4_acc: 0.9410

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 950s 336ms/step - loss: 2.7313 - dense_2_loss: 0.9471 - dense_3_loss: 0.4990 - dense_4_loss: 0.3381 - dense_2_acc: 0.8242 - dense_3_acc: 0.9152 - dense_4_acc: 0.9396 - val_loss: 0.2073 - val_dense_2_loss: 0.1731 - val_dense_3_loss: 0.0481 - val_dense_4_loss: 0.0481 - val_dense_2_acc: 0.9600 - val_dense_3_acc: 0.9899 - val_dense_4_acc: 0.9891
Epoch 13/40
2698/2825 [===========================>..] - ETA: 41s - loss: 2.7552 - dense_2_loss: 0.9551 - dense_3_loss: 0.5033 - dense_4_loss: 0.3418 - dense_2_acc: 0.8235 - dense_3_acc: 0.9157 - dense_4_acc: 0.9390

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 948s 336ms/step - loss: 2.6915 - dense_2_loss: 0.9310 - dense_3_loss: 0.4934 - dense_4_loss: 0.3361 - dense_2_acc: 0.8275 - dense_3_acc: 0.9171 - dense_4_acc: 0.9400 - val_loss: 0.3189 - val_dense_2_loss: 0.1577 - val_dense_3_loss: 0.0448 - val_dense_4_loss: 0.0451 - val_dense_2_acc: 0.9633 - val_dense_3_acc: 0.9900 - val_dense_4_acc: 0.9893
Epoch 17/40
2628/2825 [==========================>...] - ETA: 1:04 - loss: 2.7063 - dense_2_loss: 0.9351 - dense_3_loss: 0.5001 - dense_4_loss: 0.3361 - dense_2_acc: 0.8276 - dense_3_acc: 0.9168 - dense_4_acc: 0.9402

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 950s 336ms/step - loss: 2.6635 - dense_2_loss: 0.9202 - dense_3_loss: 0.4906 - dense_4_loss: 0.3325 - dense_2_acc: 0.8295 - dense_3_acc: 0.9179 - dense_4_acc: 0.9414 - val_loss: 0.1217 - val_dense_2_loss: 0.1795 - val_dense_3_loss: 0.0570 - val_dense_4_loss: 0.0481 - val_dense_2_acc: 0.9592 - val_dense_3_acc: 0.9889 - val_dense_4_acc: 0.9884
Epoch 20/40
2825/2825 [==============================] - 951s 337ms/step - loss: 2.6638 - dense_2_loss: 0.9205 - dense_3_loss: 0.4908 - dense_4_loss: 0.3319 - dense_2_acc: 0.8300 - dense_3_acc: 0.9188 - dense_4_acc: 0.9422 - val_loss: 0.2332 - val_dense_2_loss: 0.1671 - val_dense_3_loss: 0.0494 - val_dense_4_loss: 0.0464 - val_dense_2_acc: 0.9643 - val_dense_3_acc: 0.9904 - val_dense_4_acc: 0.9895
Epoch 21/40
2825/2825 [==============================] - 949s 336ms/step - loss: 2.6527 - dense_2_loss: 0.9150 - dense_3_loss: 0.4902 - dense_4_loss: 0.3325 - dense_2_acc: 0.8302 - dense_3_acc: 0.9187 - dense_4